# EFM Strong Force Analogue (S/T State, N=400)

Simulation to test the strong force analogue in the Ehokolo Fluxon Model (EFM) within the S/T state, fixing tensor construction, non-leaf tensor issues, convergence problems, and to() method errors.

## 1. Setup: Libraries, GPU Check, Drive Mount, Paths

In [ ]:
import torch
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

!nvidia-smi

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import psutil
import time
from datetime import datetime
from google.colab import drive
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"System RAM Total: {psutil.virtual_memory().total / 1e9:.2f} GB")

drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/EFM_Simulations/'
strong_path = os.path.join(base_path, 'Strong_Force_N400/')
checkpoint_path = os.path.join(strong_path, 'checkpoints/')
data_path = os.path.join(strong_path, 'data/')
plot_path = os.path.join(strong_path, 'plots/')
os.makedirs(checkpoint_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)
print(f"Paths created/checked:\n Checkpoints: {checkpoint_path}\n Data: {data_path}\n Plots: {plot_path}")

## 2. Simulation Parameters (S/T State, N=400)

In [ ]:
# --- Numerical Parameters ---
N = 400
L = 30.0
dx = L / N
max_iterations = 500
convergence_threshold = 1e-5
static_solver_iterations = 200  # Increased for better convergence
static_solver_lr = 0.001  # Reduced to stabilize optimization
save_interval = 50

# --- Physical Parameters (S/T State for Strong Force) ---
c_eff_sq = 1.0
c_eff = 1.0
m2 = 1.0
g = 10.0  # High nonlinearity for strong force binding
eta = 0.01
k_rho = 0.01
alpha = 0.1  # S/T state
q_charge = 0.0  # No EM coupling for strong force
omega_freq = np.sqrt(m2)  # Effective frequency

# --- Initial Conditions ---
struct_amplitude = 0.1  # Reduced to stabilize initial guess
struct_width = 1.0 / np.sqrt(m2) if m2 > 0 else 1.0
separations = np.linspace(struct_width * 4, L/2 * 0.8, 10)

# --- Precision ---
dtype_real = torch.float32
dtype_complex = torch.complex64

# --- Reporting ---
print("--- Strong Force Analogue Parameters ---")
print(f"Grid Size (N): {N}^3")
print(f"Target Separations: {np.round(separations, 2)}")
print(f"m^2={m2}, g={g}, eta={eta}, alpha={alpha}, q={q_charge}, omega={omega_freq:.2f}")
print(f"Precision: {dtype_real}, {dtype_complex}")
print("----------------------------------")

## 3. Helper Functions (Corrected for Strong Force)

In [ ]:
# Potential V(|phi|) = 0.5*m2*|phi|^2 - 0.25*g*|phi|^4 + 0.1667*eta*|phi|^6
def potential_V(phi_complex, m2_p, g_p, eta_p):
    phi_abs_sq = torch.abs(phi_complex)**2  # |phi|^2, real-valued
    m2_p_f32 = torch.tensor(m2_p, dtype=torch.float32, device=phi_complex.device)
    g_p_f32 = torch.tensor(g_p, dtype=torch.float32, device=phi_complex.device)
    eta_p_f32 = torch.tensor(eta_p, dtype=torch.float32, device=phi_complex.device)
    term2 = 0.5 * m2_p_f32 * phi_abs_sq
    term4 = -0.25 * g_p_f32 * phi_abs_sq**2
    term6 = (1.0/6.0) * eta_p_f32 * phi_abs_sq**3
    return term2 + term4 + term6  # Real-valued energy density

# Static Complex NLKG Solver (Energy Minimization, No A0)
def solve_nlkg_static(phi_guess, omega_p, m2_p, g_p, eta_p, c_eff_sq_p, dx_p, iterations, learning_rate, device_p, complex_dtype_p):
    # Create a leaf tensor using clone().detach()
    phi = phi_guess.clone().detach().to(dtype=complex_dtype_p, device=device_p).requires_grad_(True)
    optimizer = torch.optim.Adam([phi], lr=learning_rate)

    omega_p_t = torch.tensor(omega_p, dtype=torch.float32, device=device_p)
    c_eff_sq_p_t = torch.tensor(c_eff_sq_p, dtype=torch.float32, device=device_p)
    dx_p_t = torch.tensor(dx_p, dtype=torch.float32, device=device_p)

    print(f"Starting static solver. Initial phi max abs: {torch.max(torch.abs(phi)):.2e}")

    prev_energy = float('inf')
    for iter_num in range(iterations):
        optimizer.zero_grad()

        # --- Calculate Energy Functional ---
        # Gradient term |∇φ|²
        grad_phi_tuple = torch.gradient(phi, spacing=dx_p_t, dim=[0, 1, 2])
        grad_phi_abs_sq = sum(
            torch.abs(grad_phi_tuple[i])**2 for i in range(3)
        )
        grad_energy_density = 0.5 * c_eff_sq_p_t * grad_phi_abs_sq
        grad_energy = torch.sum(grad_energy_density) * dx_p_t**3

        # Potential term V(|φ|)
        potential_energy_density = potential_V(phi, m2_p, g_p, eta_p)
        potential_energy = torch.sum(potential_energy_density) * dx_p_t**3

        # Effective mass term (1/2)ω²|φ|² (no qA₀ term)
        phi_abs_sq = torch.abs(phi)**2
        omega_eff_sq_density = 0.5 * omega_p_t**2 * phi_abs_sq
        omega_eff_energy = torch.sum(omega_eff_sq_density) * dx_p_t**3

        # Total Energy Density (real-valued)
        total_energy_density = grad_energy_density + potential_energy_density + omega_eff_sq_density
        total_energy = torch.sum(total_energy_density) * dx_p_t**3

        # Debugging output every 50 iterations
        if iter_num % 50 == 0:
            print(f"Iter {iter_num}: Grad Energy = {grad_energy.item():.4e}, Potential Energy = {potential_energy.item():.4e}, Omega Energy = {omega_eff_energy.item():.4e}, Total Energy = {total_energy.item():.4e}")

        # Check for energy divergence
        if total_energy.item() > 1e5 or total_energy.item() > prev_energy * 10:
            print(f"Warning: Energy diverging at iteration {iter_num}. Energy: {total_energy.item():.4e}. Stopping solver.")
            break

        prev_energy = total_energy.item()

        # Backpropagate
        total_energy.backward()

        if phi.grad is None or torch.isnan(phi.grad).any() or torch.isinf(phi.grad).any():
            print(f"Warning: Invalid gradient at iteration {iter_num}. Stopping solver.")
            if 'phi_prev' in locals():
                with torch.no_grad():
                    phi.copy_(phi_prev)
                optimizer = torch.optim.Adam([phi], lr=learning_rate)
                print("Restored previous state.")
            else:
                print("Cannot restore state, stopping.")
            break

        # Gradient clipping to prevent instability
        torch.nn.utils.clip_grad_norm_(phi, max_norm=1.0)

        phi_prev = phi.detach().clone()
        optimizer.step()

        with torch.no_grad():
            phi.real.clamp_(-50.0, 50.0)
            phi.imag.clamp_(-50.0, 50.0)

        # Cleanup
        del grad_phi_tuple, grad_phi_abs_sq, grad_energy_density, potential_energy_density, omega_eff_sq_density, total_energy_density, grad_energy, potential_energy, omega_eff_energy, total_energy
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # --- Final Energy Calculation ---
    final_phi = phi.detach().clone()
    with torch.no_grad():
        grad_phi_tuple = torch.gradient(final_phi, spacing=dx_p_t, dim=[0, 1, 2])
        grad_phi_abs_sq = sum(
            torch.abs(grad_phi_tuple[i])**2 for i in range(3)
        )
        grad_energy_density = 0.5 * c_eff_sq_p_t * grad_phi_abs_sq
        potential_energy_density = potential_V(final_phi, m2_p, g_p, eta_p)
        phi_abs_sq = torch.abs(final_phi)**2
        omega_eff_sq_density = 0.5 * omega_p_t**2 * phi_abs_sq
        total_energy_density = grad_energy_density + potential_energy_density + omega_eff_sq_density
        final_energy = torch.sum(total_energy_density).item() * dx_p_t.item()**3

    print(f"Static solver finished. Final energy: {final_energy:.4e}, Final phi max abs: {torch.max(torch.abs(final_phi)):.2e}")

    # Cleanup
    del phi, optimizer, grad_phi_tuple, grad_phi_abs_sq, grad_energy_density, potential_energy_density, omega_eff_sq_density, total_energy_density
    if 'phi_prev' in locals():
        del phi_prev
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return final_phi, final_energy

# Calculate interaction energy (No A0)
def calculate_interaction(phi_complex, omega_p, m2_p, g_p, eta_p, c_eff_sq_p, dx_p, single_energy, device_p):
    with torch.no_grad():
        omega_p_t = torch.tensor(omega_p, dtype=torch.float32, device=device_p)
        c_eff_sq_p_t = torch.tensor(c_eff_sq_p, dtype=torch.float32, device=device_p)
        dx_p_t = torch.tensor(dx_p, dtype=torch.float32, device=device_p)

        grad_phi_tuple = torch.gradient(phi_complex, spacing=dx_p_t, dim=[0, 1, 2])
        grad_phi_abs_sq = sum(
            torch.abs(grad_phi_tuple[i])**2 for i in range(3)
        )
        grad_energy_density = 0.5 * c_eff_sq_p_t * grad_phi_abs_sq
        potential_energy_density = potential_V(phi_complex, m2_p, g_p, eta_p)
        phi_abs_sq = torch.abs(phi_complex)**2
        omega_eff_sq_density = 0.5 * omega_p_t**2 * phi_abs_sq

        total_energy_density = grad_energy_density + potential_energy_density + omega_eff_sq_density
        total_energy = torch.sum(total_energy_density).item() * dx_p_t.item()**3

        single_energy_val = float(single_energy)
        interaction_energy = total_energy - 2 * single_energy_val

        del grad_phi_tuple, grad_phi_abs_sq, grad_energy_density, potential_energy_density, omega_eff_sq_density, total_energy_density
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return interaction_energy, total_energy

# Create initial guess (two separated structures)
def create_initial_guess(separation, amp, width, N_p, L_p, dtype_real_p, dtype_complex_p, device_p):
    x_coord = torch.linspace(-L_p/2, L_p/2, N_p, device=device_p, dtype=dtype_real_p)
    X, Y, Z = torch.meshgrid(x_coord, x_coord, x_coord, indexing='ij')

    def profile(X0, Y0, Z0, width_p):
        R_sq = (X - X0)**2 + (Y - Y0)**2 + (Z - Z0)**2
        return amp * torch.exp(-R_sq / width_p**2)

    phi1_real = profile(-separation / 2.0, 0, 0, width)
    phi2_real = profile(separation / 2.0, 0, 0, width)
    phi_real_guess = phi1_real + phi2_real

    phi_guess = torch.complex(phi_real_guess, torch.zeros_like(phi_real_guess)).to(dtype_complex_p)

    del X, Y, Z, x_coord, phi1_real, phi2_real, phi_real_guess
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return phi_guess

## 4. Iterative Solver Loop for Different Separations

In [ ]:
print("\n--- Starting Strong Force Calculation --- ")
overall_start_time = time.time()

# Calculate single ehokolon energy
print("Calculating single ehokolon energy...")
E_single = None
phi_single_stable = None
try:
    phi_single_guess = create_initial_guess(L, struct_amplitude, struct_width, N, L, dtype_real, dtype_complex, device)
    phi_single_stable, E_single = solve_nlkg_static(
        phi_single_guess.detach(), omega_freq, m2, g, eta, c_eff_sq, dx,
        static_solver_iterations, static_solver_lr, device, dtype_complex
    )
    print(f"Single ehokolon energy E_single = {E_single:.4e}")
    del phi_single_guess
except Exception as e:
    print(f"Error calculating single ehokolon energy: {e}")
    E_single = float('nan')

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Store results
interaction_energies = []
total_energies = []
final_phis = {}

if not np.isnan(E_single):
    for i, sep in enumerate(tqdm(separations, desc="Separations")):
        print(f"\n--- Running for separation d = {sep:.2f} ---")
        iter_start_time = time.time()

        phi_current = create_initial_guess(sep, struct_amplitude, struct_width, N, L, dtype_real, dtype_complex, device)
        phi_converged = False

        # Iterative loop for phi optimization
        for iter_num in tqdm(range(max_iterations), desc=f"d={sep:.2f} Iter", leave=False):
            phi_old = phi_current.detach().clone()

            phi_current_opt, _ = solve_nlkg_static(
                phi_old.detach(), omega_freq, m2, g, eta, c_eff_sq, dx,
                static_solver_iterations//10, static_solver_lr, device, dtype_complex
            )
            phi_current = phi_current_opt.detach().clone()

            # Check convergence
            change = torch.norm(phi_current - phi_old).item() / (torch.norm(phi_old).item() + 1e-9)
            if iter_num % 50 == 0:
                print(f"  Iter {iter_num}, Change: {change:.2e}")
            if change < convergence_threshold:
                print(f" Converged after {iter_num+1} iterations for d={sep:.2f}.")
                phi_converged = True
                break
            del phi_old

        if not phi_converged:
            print(f" Warning: Did not converge after {max_iterations} iterations for d={sep:.2f}.")

        # Calculate final energies
        E_interaction, E_total = calculate_interaction(
            phi_current, omega_freq, m2, g, eta, c_eff_sq, dx, E_single, device
        )

        interaction_energies.append(E_interaction)
        total_energies.append(E_total)

        final_phis[sep] = phi_current[N//2, :, :].cpu().numpy().astype(np.complex64)

        print(f" d={sep:.2f}: E_total={E_total:.4e}, E_interaction={E_interaction:.4e}")

        del phi_current
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        iter_end_time = time.time()
        print(f"--- Iteration for d = {sep:.2f} finished in {(iter_end_time - iter_start_time):.2f} seconds ---")
else:
    print("Skipping interaction calculation due to failure in single ehokolon energy calculation.")

# Save results
try:
    save_dict = {
        'separations': np.array(separations, dtype=np.float32),
        'interaction_energies': np.array(interaction_energies, dtype=np.float32),
        'total_energies': np.array(total_energies, dtype=np.float32),
        'E_single': np.array(E_single, dtype=np.float32) if E_single is not None else np.array(float('nan')),
        'final_phi_slices': final_phis
    }
    npz_file_path = os.path.join(data_path, f"strongforce_results_N{N}.npz")
    np.savez(npz_file_path, **save_dict)
    print(f"Saved final analysis results to {npz_file_path}")
except Exception as e:
    print(f"Error saving final results: {e}")

# Cleanup
if 'phi_single_stable' in locals():
    del phi_single_stable
del final_phis, interaction_energies, total_energies
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

overall_end_time = time.time()
print(f"\n=== Strong Force Simulation finished in {(overall_end_time - overall_start_time) / 60:.2f} minutes ===")

## 5. Analysis and Visualization (Strong Force)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.optimize import curve_fit

# Load results
results_file = os.path.join(data_path, f"strongforce_results_N{N}.npz")
results_strong = None
if os.path.exists(results_file):
    try:
        data = np.load(results_file, allow_pickle=True)
        results_strong = {key: data[key].item() if data[key].ndim == 0 and data[key].dtype != object else data[key]
                         for key in data.files}
        if 'final_phi_slices' in results_strong and isinstance(results_strong['final_phi_slices'], np.ndarray):
            results_strong['final_phi_slices'] = results_strong['final_phi_slices'].item()
        print(f"Loaded Strong Force results from {results_file}")
    except Exception as e:
        print(f"Error loading file {results_file}: {e}")
else:
    print(f"Result file not found: {results_file}")

if results_strong and 'separations' in results_strong and len(results_strong['separations']) > 0:
    separations_an = results_strong['separations']
    interaction_energies_an = results_strong['interaction_energies']

    valid_energy_idx = np.isfinite(interaction_energies_an)
    if valid_energy_idx.sum() > 1:
        forces = np.full_like(separations_an, float('nan'))
        forces[valid_energy_idx] = -np.gradient(interaction_energies_an[valid_energy_idx], separations_an[valid_energy_idx])
        valid_force_idx = np.isfinite(forces)
    else:
        print("Warning: Cannot calculate forces due to insufficient or invalid energy data.")
        forces = np.full_like(separations_an, float('nan'))
        valid_force_idx = np.isfinite(forces)

    # Plot Interaction Energy and Force
    plt.figure(figsize=(14, 6))

    # Interaction Energy
    ax1 = plt.subplot(1, 2, 1)
    if valid_energy_idx.any():
        ax1.plot(separations_an[valid_energy_idx], interaction_energies_an[valid_energy_idx], 'bo-', label='E_interaction (Simulated)')
        valid_fit_idx_E = valid_energy_idx & (separations_an > 0)
        if valid_fit_idx_E.sum() > 1:
            try:
                def energy_func(r, A, C):
                    return -A * np.exp(-r / 1.0) + C  # Yukawa-like potential for strong force
                popt_e, _ = curve_fit(energy_func, separations_an[valid_fit_idx_E], interaction_energies_an[valid_fit_idx_E])
                fit_label_E = f'Yukawa-like fit (A={popt_e[0]:.2e})'
                ax1.plot(separations_an[valid_fit_idx_E], energy_func(separations_an[valid_fit_idx_E], *popt_e), 'r--', label=fit_label_E, alpha=0.7)
            except Exception as fit_e_err:
                print(f"Could not fit Yukawa to energy: {fit_e_err}")

    ax1.set_xlabel("Separation d")
    ax1.set_ylabel("Interaction Energy E_int")
    ax1.set_title("Interaction Energy vs Separation (S/T)")
    ax1.grid(True, which="both", ls="--")
    ax1.legend()

    # Force
    ax2 = plt.subplot(1, 2, 2)
    if valid_force_idx.any():
        ax2.plot(separations_an[valid_force_idx], forces[valid_force_idx], 'go-', label='Force (-dE/dr, Sim.)')
        valid_fit_idx_F = valid_force_idx & (separations_an > 0)
        if valid_fit_idx_F.sum() > 1:
            try:
                def force_func(r, B, D):
                    return B * (np.exp(-r / 1.0) / r) + D  # Yukawa-like force
                popt_f, _ = curve_fit(force_func, separations_an[valid_fit_idx_F], forces[valid_force_idx_F])
                force_type = "Attractive" if popt_f[0] > 0 else "Repulsive"
                fit_label_F = f'Yukawa-like fit ({force_type}, B={popt_f[0]:.2e})'
                ax2.plot(separations_an[valid_fit_idx_F], force_func(separations_an[valid_fit_idx_F], *popt_f), 'm--', label=fit_label_F, alpha=0.7)
            except Exception as fit_f_err:
                print(f"Could not fit Yukawa to force: {fit_f_err}")

    ax2.set_xlabel("Separation d")
    ax2.set_ylabel("Force F = -dE/dr")
    ax2.set_title(f"Force vs Separation (S/T, N={N})")
    ax2.grid(True, which="both", ls="--")
    ax2.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(plot_path, f"strongforce_N{N}_energy_force.png"))
    plt.show()

    # Visualize Final Fields
    final_phi_slices_dict = results_strong.get('final_phi_slices', {})
    if final_phi_slices_dict and len(separations_an) > 0:
        plot_idx = len(separations_an) // 2
        sep_to_plot = separations_an[plot_idx]
        actual_phi_key = min(final_phi_slices_dict.keys(), key=lambda k: abs(k - sep_to_plot))
        phi_slice = final_phi_slices_dict.get(actual_phi_key)

        if phi_slice is not None:
            plt.figure(figsize=(6, 5))
            im_phi = plt.imshow(np.abs(phi_slice).astype(np.float32), extent=[-L/2, L/2, -L/2, L/2], cmap='viridis', aspect='auto')
            plt.colorbar(im_phi)
            plt.title(f"Final |φ| (z=0) for d≈{sep_to_plot:.2f}")
            plt.xlabel("x")
            plt.ylabel("y")
            plt.tight_layout()
            plt.savefig(os.path.join(plot_path, f"strongforce_N{N}_final_field_sep{sep_to_plot:.1f}.png"))
            plt.show()
        else:
            print(f"Slice data not found for separation key near {sep_to_plot}")
else:
    print("No results loaded to analyze.")

## 6. Simulation Report (Strong Force)

In [ ]:
print("\n--- Strong Force Simulation Report ---")
print(f"Simulation Timestamp: {datetime.now()}")
print(f"Grid Size (N): {N}")
print(f"Box Size (L): {L}")

if results_strong:
    print(f"Tested separations: {np.round(results_strong.get('separations', []), 2)}")
    print(f"Calculated interaction energies: {results_strong.get('interaction_energies', [])}")
    print("See generated plots for Energy vs Separation and Force vs Separation trends.")
    if 'forces' in locals() and np.isfinite(forces).all():
        print("Force calculation seemed stable and followed expected trend (check plots).")
    elif 'forces' in locals():
        print("Force calculation completed but might contain invalid values (check plots).")
    else:
        print("Force calculation was likely skipped due to insufficient/invalid energy data.")
else:
    print("No simulation results loaded to report.")

print("\nAnalysis plots potentially saved to:", plot_path)
print("Data files potentially saved to:", data_path)
print("Checkpoints potentially saved to:", checkpoint_path)
print("----------------------------------")